In [432]:
from gamspy import Container
from gamspy import Model
from gamspy import Set
from gamspy import Parameter
from gamspy import Variable
from gamspy import Equation
from gamspy import Sum

import pandas as pd
import numpy as np

import os

## DATA DEFINITION

In [433]:
data_path = "data"

# Set i
products_info = pd.read_excel(os.path.join(data_path, "Products.xlsx"), sheet_name="Sayfa1", index_col=None)
products = [i.replace(" ", "") for i in products_info.Products.tolist()]
set_i = products

# Set j
distance = pd.read_excel(os.path.join(data_path, "distance-matrix.xlsx"), sheet_name="Sayfa1", index_col=0).fillna(0)
cities = [i.replace("İ", "i").lower() for i in distance.columns]
distance.columns = cities
distance.index = cities
set_j = cities

# Set k
ports = pd.read_excel(os.path.join(data_path, "Limanlar.xlsx"), sheet_name="Sayfa1", index_col=None)
ports = [i.replace("İ", "i").lower() for i in ports.PortName.to_list()]
set_k = ports


In [434]:
# Parametes

# w
forecasts = pd.read_csv(os.path.join(data_path, "export_forecasts.csv"), index_col=0)
param_w = np.array(forecasts["2024"])

# d
param_d = distance[ports].T.unstack()

# q
quality = pd.read_excel(os.path.join(data_path, "Quality_ProdCap.xlsx"), sheet_name="Quality", index_col=0).fillna(0).T
quality.columns = [i.replace("İ", "i").lower() for i in quality.columns]
quality.index = [i.replace(" ", "").lower() for i in quality.index]
param_q = quality[cities].T.unstack()

# v
param_v = np.array([int(i.split()[0]) for i in products_info["Quantity (1 Container)"]])

# cap
product_capacity = pd.read_excel(os.path.join(data_path, "Quality_ProdCap.xlsx"), sheet_name="ProdCap", index_col=0).fillna(0).T
product_capacity.columns = [i.replace("İ", "i").lower() for i in product_capacity.columns]
product_capacity.index = [i.replace(" ", "").lower() for i in product_capacity.index]
param_cap = product_capacity[cities].T.unstack()

# wn
param_wn = 1       # Userinput

# max_cap
param_mc = 1000      # Userinput

# warehouse_cost
warehouse_house = pd.read_excel(os.path.join(data_path, "warehouse_cost.xlsx"), sheet_name="Sheet1", index_col=None)
param_wc = np.array(warehouse_house["cost"])*30

# revenue
param_revenue = np.array(products_info["Revenue (1 Container - $)"])

# budget
param_mb = 1000000     # Userinput

# transportation_cost
param_trans_cost = distance[ports].T.unstack()*2


## BUILD MODELS

In [435]:
# Create a container
m = Container()


### SETS

In [436]:
# Create the model sets
i = Set(m, name='i', description='Products considered', records=set_i)
j = Set(m, name='j', description='Potential warehouse locations', records=set_j)
k = Set(m, name='k', domain=j, description='Exporting Hubs (Ports)', records=set_k)


### PARAMETERS

In [437]:
# Create the model parameters

w = Parameter(m, name='w', domain=i, description='Weights for product i', records=param_w)

d = Parameter(m, name='d', domain=[j, k], description='Distance from city j to exporting hub k', records=param_d)

q = Parameter(m, name='q', domain=[i, j], description='Quality of product i in city j', records=param_q)

v = Parameter(m, name='v', domain=i, description='Volume of product i', records=param_v)

p_cap = Parameter(m, name='p_cap', domain=[i, j], description='Production Capacity of product i in city j', records=param_cap)

wn = Parameter(m, name='wn', description='Maximum number of warehouses', records=param_wn)

m_cap = Parameter(m, name='m_cap', description='Maximum capacity of warehouses', records=param_mc)

wc = Parameter(m, name='wc', domain=j, description='Warehouse cost per container per city', records=param_wc)

revenue = Parameter(m, name='revenue', domain=i, description='Revenue per container per product', records=param_revenue)

m_budget = Parameter(m, name='m_budget', description='Maximum budget for warehouses', records=param_mb)

transport_cost = Parameter(m, name="transport_cost", domain=[j, k], description='Transportation cost from city j to exporting hub k', records=param_trans_cost)

### VARIABLES

In [438]:
# Create the model variables
x = Variable(m, name='x', domain=j, type="binary", description='Whether to build a warehouse in city j')
y = Variable(m, name='y', domain=[i, j], type="positive", description='Amount of product i stocked at city j')

o1 = Variable(m, name='o1', type="free", description='Variable to take first objective function value')
o2 = Variable(m, name='o2', type="free", description='Variable to take second objective function value')
o3 = Variable(m, name='o3', type="free", description='Variable to take third objective function value')
o4 = Variable(m, name='o4', type="free", description='Variable to take fourth objective function value')

o_all = Variable(m, name='o_all', type="free", description='Variable to take the total objective function value')

### EQUATIONS

In [439]:
# Define the model equations
obj1 = Equation(m, name='obj1', description='Maximize the total products value')
obj2 = Equation(m, name='obj2', description='Minimize the total distance')
obj3 = Equation(m, name='obj3', description='Maximize the total products quality')
obj4 = Equation(m, name='obj4', description='Maximize the total profit')

weighted_obj = Equation(m, name='weighted_obj', description='Weighted objective function')

num_warehouses = Equation(m, name='num_warehouses', description='Number of warehouses built')

store_if_built = Equation(m, name='store_if_built', domain=[i, j], description='Amount of product i stored at city j if a warehouse is built')

budget = Equation(m, name='budget', description='Budget constraint')

capacity = Equation(m, name='capacity', domain=j, description='Capacity constraint')


In [440]:
# Maximize the total products value
obj1[...] = o1 == Sum([i, j], w[i] * y[i, j])

# Minimize the total distance
obj2[...] = o2 == Sum([j, k], d[j, k] * x[j])

# Maximize the total products quality
obj3[...] = o3 == Sum([i, j], (q[i, j] + p_cap[i, j]) * x[j])

# Maximize the total profit
obj4[...] = o4 == Sum([i, j, k], (revenue[i] - transport_cost[j, k]) * y[i, j]) - Sum(j, wc[j] * x[j])

# Weighted objective function
weighted_obj[...] = o_all == (1*o1) - (10000*o2) + (100000*o3) + (10*o4)


# CONSTRAINTS

# Number of warehouses built
num_warehouses[...] = Sum(j, x[j]) <= wn

# Amount of product i stored at city j if a warehouse is built
store_if_built[i, j] = y[i, j] <= x[j] * 1000000000000

# Budget constraint
budget[...] = Sum(j, x[j] * wc[j]) <= m_budget *1000

# Capacity constraint
capacity[j] = Sum(i, y[i, j]*v[i]) <= m_cap

In [441]:
scm = Model(m, name="scm", equations=m.getEquations(), problem="MIP", sense="MAX", objective=o_all)
scm.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,366956999.98433,487,485,MIP,CPLEX,0.047


In [442]:
o1.toValue(), o2.toValue(), o3.toValue(), o4.toValue(), o_all.toValue(), param_w

(3208350.4166666665,
 5670.0,
 2006.427783812222,
 21980587.118644066,
 366956999.9843296,
 array([1.39477877e+06, 2.02133500e+04, 6.09897920e+05, 2.67317650e+05,
        8.77007300e+04, 2.94224765e+06, 1.87330480e+05, 4.41052790e+05,
        7.21674700e+04, 3.51234100e+04, 8.81298600e+04, 7.70004100e+04,
        1.42720530e+05, 2.43724000e+03, 2.94224765e+06]))

In [443]:
warehouses = x.records.j[x.records.level == 1].tolist()
y.pivot()[warehouses]


,istanbul
T-shirts,0.000000
BathTowel,0.000000
Table&Chair,0.000000
GardenFur.,0.000000
Beds,0.000000
Carpets,0.000000
SteelDoor,0.000000
Detergant,0.000000
Bicycle,0.000000
OutdoorCloths,0.000000
